## 영화 리뷰 분류 : 이진 분류 문제
2종 분류 또는 이진분류는 아마도 가장 널리 적용된 머신러닝 문제일 것이다. IMDB 데이터 셋의 리뷰 텍스트를 기반으로 긍정과 부정으로 분류하는 방법을 배워보자

In [42]:
# 데이터 셋 로드
from tensorflow.keras.datasets import imdb

# num_words = 10000 : 훈련 데이터에서 가장 자주 나타나는 단어 1만 개만 사용하겠다는 의미
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

17464789/17464789 [==============================] - 0s 0us/step


In [43]:
# 데이터 셋 로드
from tensorflow.keras.datasets import imdb

# num_words = 10000 : 훈련 데이터에서 가장 자주 나타나는 단어 1만 개만 사용하겠다는 의미
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = 10000)

In [44]:
max([max(sequence) for sequence in train_data])

9999

신경망에 숫자 리스트를 바로 주입할 수는 없다. 숫자 리스트 자체는 모두 길이가 다르지만 신경망은 동일한 크기의 배치를 기대하기 때문이다. 리스트를 텐서로 바꾸는 두 가지 방법은
1. 같은 길이가 되도록 리스트에 패딩을 추가하고 (samples, max_length) 크기의 정수 텐서로 변환한다. 그 다음 이 정수 텐서를 다룰 수 있는 층으로 신경망을 시작
2. 리스트를 멀티-핫 인코딩(multi-hot encoding) 하여 0과 1의 벡터로 변환한다. 예를 들어 시퀸스 [8, 5] 를 인덱스 8과 5의 위치는 1이고 그 외는 모두 0인 10,000 차원의 벡터로 각각 변환하는 것이다. 그 다음 부동 소수점 벡터 데이터를 다룰 수 있는 Dense 층을 신경망의 첫 번째 층으로 사용한다.

In [45]:
# 멀티-핫 인코딩으로 인코딩하기
import numpy as np

def vectorize_sequences(sequences, dimension=10000):
  # 길이가 ((len(sequences), dimension))이고 모든 원소가 0인 행렬을 만든다
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences) :
    for j in sequence :
      # results[i]에서 특정 인덱스의 위치를 1로 만든다
      results[i, j] = 1
  return results

# 훈련 데이터를 벡터로 변환
x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

In [46]:
print('x_train[0] : ', x_train[0])
print('x_train : ', x_train)
print('x_train.shape : ', x_train.shape)

x_train[0] :  [0. 1. 1. ... 0. 0. 0.]
x_train :  [[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 ...
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]
 [0. 1. 1. ... 0. 0. 0.]]
x_train.shape :  (25000, 10000)


In [47]:
# 레이블 벡터로 변환
y_train = np.asarray(train_labels).astype("float32")
y_test = np.asarray(test_labels).astype("float32")

신경망 모델 만들기<br>
입력 데이터가 벡터고 레이블은 스칼라(1 or 0)이다. 이런 문제가 잘 작동하는 모델은 relu 활성화 함수를 사용한 밀집 연결 층을 그냥 쌓은 것이다.<br><br>
Dense 층을 쌓을 때는 두 가지 중요한 구조상의 결정이 필요한데
- 얼마나 많은 층을 사용할 것인가?
- 각 층에 얼마나 많은 유닛을 둘 것인가?


In [48]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Dense

model = keras.Sequential([
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

모델의 끝에 시그모이드 활성화 함수를 사용한 하나의 유닛으로 된 층을 놓은 것은 이 문제가 이진 분류 문제이고 모델의 출력이 확률이기 때문이다. 따라서 손실 함수 역시 `binary_crossentropy` 가 적합하다. 확률을 출력하는 모델을 사용할 때는 크로스 엔트로피가 최선의 선택이라 말할 수 있다. (단, 이 함수가 유일한 선택지는 아니고 `mean_squared_error`를 이용할 수도 있다)
<br><br>
크로스 엔트로피(crossentropy)는 정보이론(information theory)분야에서 온 개념으로 확률 분포 간의 차이를 측정하며, 여기서는 원본 분포와 예측 분포 사이를 측정한다.

In [49]:
# 손실함수와 옵티마이저 선택
# 옵티마이저 rmsprop는 일반적으로 거의 모든 문제에 기본 선택으로 고려할만큼 좋다.
model.compile(optimizer = 'rmsprop',
              loss = 'binary_crossentropy',
              metrics = 'acc')

In [50]:
# 딥러닝 모델은 훈련 데이터에서 평가해서는 안되기 때문에, 원본 훈련 데이터에서 1만 개의 샘플을 떼어내 검증 세트를 만들어보자
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
# y_val = train_labels[:10000]
# partial_y_train = train_labels[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

이제 샘플을 512개씩 미니배치하여 20번의 epochs 동안 돌려(훈련 시켜)보자. 동시에 분리시켜줬던 1만 개의 검증 세트로 손실과 정확도를 측정할 것이다.

In [ ]:
# fit 정보(반환되는 History 객체)를 저장하기 위해 history 변수를 설정해주었다
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs = 20,
    batch_size = 512,
    validation_data = (x_val, y_val)
)

![image](https://user-images.githubusercontent.com/51469989/212635964-867b6b6a-6225-4ff4-9d44-2909db1d1e54.png)

In [ ]:
# History 객체는 훈련하는 동안 모든 정보를 담고 있는 딕셔너리인 history 속성을 가지고 있다.
# 이것이 저장된 history 변수로 확인해보자
print('history.history.keys() : ', history.history.keys())
print('history.history["loss"] : ', history.history['loss'])
print('history.history["acc"] : ', history.history['acc'])

```
history.history.keys() :  dict_keys(['loss', 'acc', 'val_loss', 'val_acc'])
history.history["loss"] :  [0.5068102478981018, 0.3009280562400818, 0.22127921879291534, 0.17036522924900055, 0.13541054725646973, 0.11182797700166702, 0.09349296987056732, 0.07766947150230408, 0.06217286363244057, 0.05161004513502121, 0.04238727688789368, 0.03334704041481018, 0.02928905375301838, 0.020846325904130936, 0.013812199234962463, 0.01781359128654003, 0.007585851475596428, 0.009006377309560776, 0.011024264618754387, 0.0033360531087964773]
history.history["acc"] :  [0.782800018787384, 0.9004666805267334, 0.9265999794006348, 0.9421333074569702, 0.9564666748046875, 0.9656000137329102, 0.973800003528595, 0.9793333411216736, 0.9840666651725769, 0.987333357334137, 0.990066647529602, 0.9935333132743835, 0.9935333132743835, 0.9969333410263062, 0.9990000128746033, 0.9970666766166687, 0.9995999932289124, 0.9991333484649658, 0.9974666833877563, 0.9998666644096375]
```

In [ ]:
# 위를 보면 훈련시 손실-정확도 값과 검증시 손실-정확도 값이 저장되어 있는 것을 알 수 있다.
# 이를 이용해 훈련과 검증 손실 그래프를 그려보자
import matplotlib.pyplot as plt

history = history.history

fig, axs = plt.subplots(1, 2, figsize=(20, 10))
axs = axs.flatten()

acc = history['acc']
val_acc = history['val_acc']
loss = history['loss']
val_loss = history['val_loss']

epochs = range(len(acc))

axs[0].plot(epochs, loss, 'r', label='Training loss')
axs[0].plot(epochs, val_loss, 'b', label='Validation loss')
axs[0].set_title('Training and validation loss')
axs[0].legend(loc=0)

axs[1].plot(epochs, acc, 'r', label='Training accuracy')
axs[1].plot(epochs, val_acc, 'b', label='Validation accuracy')
axs[1].set_title('Training and validation accuracy')
axs[1].legend(loc=0)

![image](https://user-images.githubusercontent.com/51469989/212636114-1dcd5d26-6999-4567-b749-1fce7c1403f8.png)

Epochs가 진행될 수록 훈련 손실함수가 감소하고, 훈련 정확도는 증가하는 모습을 확인할 수 있었다. 하지만 검증의 경우 훈련과는 다른 모습이 보인다. 이는 과대적합(overfitting) 되었기 때문이라고 생각할 수도 있다. 훈련 데이터에 과도하게 최적화되어 훈련 세트 이외의 데이터에는 일반화되지 못할 수 있는 것이다.
<br><br>
이런 경우 overfitting을 방지하게 위해 특정 에폭 이후에 훈련을 중지할 수도 있습니다. 처음부터 다시 새로운 신경망을 4번의 epochs 동안만 훈련하고 평가해보자

In [ ]:
history = model.fit(x_train, y_train, epochs = 4, batch_size = 512)

![image](https://user-images.githubusercontent.com/51469989/212636217-144429fd-fdd6-4e0d-8f1b-ba1eb7f8e97b.png)

In [ ]:
# 테스트 손실, 테스트 정확도 추출
model.evaluate(x_test, y_test)

```
782/782 [==============================] - 2s 2ms/step - loss: 0.5622 - acc: 0.8514
[0.5621730089187622, 0.8514000177383423]
```

In [ ]:
# 확률 예측
print(model.predict(x_test))

```
782/782 [==============================] - 3s 3ms/step
[[0.00388645]
 [0.99999875]
 [0.904732  ]
 ...
 [0.03210984]
 [0.0519547 ]
 [0.9455671 ]]
 ```